# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re, operator, itertools
import datetime
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Iterable
import pandas as pd

# Defining repository and directories

In [2]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/pig/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_pig/'

# Loading comparison data between bugline and blame

In [3]:
compare_dirs = analyze_dir + '04_annotate/02_diff-file_blame-file_comparison/'

In [4]:
dirs = []
for d in glob.iglob(compare_dirs + '*', recursive=True):
    dirs.append(d)
dirs.sort()
dirs

['/Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/histogram_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/minimal_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/myers_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/patience_comparison']

In [5]:
column_name = ['bug_id','line_number','buggy_code','does_bugline_from_originfile?','date_added_in_originfile','earliest_affected_version','version_release_date','bug_class']

In [12]:
paths = []
for d in glob.iglob(dirs[1] + "/*", recursive=True):
    paths.append(d)
paths[21]

'/Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/minimal_comparison/diff_TezOperPlan.java_minimal-blame_TezOperPlan.java_1a2b4f3a0-0786828e27_PIG-5132_621.csv'

# Reading files from comparison dataset

In [14]:
#Reading files from comparison
for n, folder in enumerate(dirs):
    result = []
    z = 1
    for file in glob.iglob(folder + '/*', recursive=True):
        sys.stdout.write("\r{} Reading file no: {}".format((n+1),z))
        sys.stdout.flush()
        z += 1
        res = []
        with open(file,'r') as myfile:
            csv_reader = csv.reader(myfile, delimiter=',')
            for row in csv_reader:
                res.append(row[7])
        del res[0]

        regex = re.search("(?<=\/diff\_)(?P<filename>.\w+[a-zA-Z0-9-.]+(?=_))\_(?P<algorithm>\w+)\-.*\_(?P<commitid>[0-9a-z]{9})\-(?P<parentid>[0-9a-z]{10})\w.*\_(?P<filenumber>\d+)", file)
        regex = regex.groups()

        if ('bug-introducing change') not in res:
            tmp = [file,row[0],regex[0],regex[2],regex[3],int(regex[4]),"Does not affect the version"]
        else:
            tmp = [file,row[0],regex[0],regex[2],regex[3],int(regex[4]),"Affect the version"]
        result.append(tmp)
    result = sorted(result, key=operator.itemgetter(2,0))
    
    with open(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/" + regex[1] + "_affected_files.csv",'w') as csvfile:
        fields = ['file','bug_id','filename','bugintro_commitID','parent_id','filenumber','affect_version?']
        writers = csv.writer(csvfile, delimiter=",")
        writers.writerow(fields)
        for item in result:
            writers.writerow(item)
            
print ("\nThe process is complete")

4 Reading file no: 708 --> /Users/YusufNugroho/different-diff/analyze/analyze_pig/04_annotate/02_diff-file_blame-file_comparison/patience_comparison/diff_TEZC-Union-2.gld_patience-blame_TEZC-Union-2.gld_5c0a7d08a-901649025c_PIG-4737_476.csv17.csv_490.csvsvG-5132_542.csv18.csvsv4.csvsvder_1a2b4f3a0-0786828e27_PIG-5132_689.csvsv
The process is complete


In [ ]:
df_affhist = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/histogram_affected_files.csv")
df_affhist = df_affhist[fields]
df_affhist

In [ ]:
df_affmin = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/minimal_affected_files.csv")
df_affmin = df_affmin[fields]
df_affmin

In [ ]:
df_affmyers = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/myers_affected_files.csv")
df_affmyers = df_affmyers[fields]
df_affmyers

In [ ]:
df_affpat = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/patience_affected_files.csv")
df_affpat = df_affpat[fields]
df_affpat

# Counting affected and non-affected files

In [ ]:
df = [df_affhist,df_affmin,df_affmyers,df_affmyers]

In [ ]:
algorithms = ['histogram','minimal','myers','patience']

In [ ]:
no_of_files = []
for z, dframe in enumerate(df):
    affnum = dframe['affect_version?'][dframe['affect_version?'] == 'Does not affect the version'].count()
    nonaffnum = dframe['affect_version?'][dframe['affect_version?'] == 'Affect the version'].count()
    summ = [algorithms[z],affnum,nonaffnum]
    no_of_files.append(summ)

In [ ]:
with open(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/totalnumber_of_affectedfiles.csv",'w') as file:
    header = ['algorithm','#non-affected_files','#affected_files']
    writers = csv.writer(file,delimiter=",")
    writers.writerow(header)
    for ii in no_of_files:
        writers.writerow(ii)
print ("File 'totalnumber_of_affectedfiles.csv' has been created")

In [ ]:
aff_file = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/totalnumber_of_affectedfiles.csv")
aff_file